In [0]:
  !wget https://archive.ics.uci.edu/ml/machine-learning-databases/00222/bank-additional.zip


In [0]:
from tqdm import tqdm
# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

In [28]:
!unzip /content/bank-additional.zip

Archive:  /content/bank-additional.zip
replace bank-additional/.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [0]:
import pandas as pd
#df = pd.read_csv('/content/bank-additional/bank-additional.csv',sep=';')
#df = pd.read_csv('/content/Iris_processed.csv')

In [30]:
import pandas as pd
from sklearn.utils import shuffle
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

'''def load_data():
  
  temp = pd.read_csv('/content/bank-additional/bank-additional-full.csv',sep=';')
  #df.info()
  df=temp[['pdays','euribor3m','cons.price.idx','cons.conf.idx','age','duration','nr.employed','y']]
  df=shuffle(df)
  Y=df['y']
  Y=Y.eq('yes').mul(1)
  df=df.drop(['y'],axis=1)
  #df=pd.get_dummies(df)
  sm = SMOTE(random_state=42)
  X_res, y_res = sm.fit_resample(df, Y)
  X=pd.DataFrame(X_res,columns=df.columns)
  Y=pd.DataFrame(y_res,columns=['y'])
  #print(X_res)
  #print(y_res)
  X,testX,Y,testY = train_test_split(X,Y,test_size=0.7,stratify=Y)
  #temp1=temp[0:8000]
  #print(temp1)
  return X,Y
  '''

"def load_data():\n  \n  temp = pd.read_csv('/content/bank-additional/bank-additional-full.csv',sep=';')\n  #df.info()\n  df=temp[['pdays','euribor3m','cons.price.idx','cons.conf.idx','age','duration','nr.employed','y']]\n  df=shuffle(df)\n  Y=df['y']\n  Y=Y.eq('yes').mul(1)\n  df=df.drop(['y'],axis=1)\n  #df=pd.get_dummies(df)\n  sm = SMOTE(random_state=42)\n  X_res, y_res = sm.fit_resample(df, Y)\n  X=pd.DataFrame(X_res,columns=df.columns)\n  Y=pd.DataFrame(y_res,columns=['y'])\n  #print(X_res)\n  #print(y_res)\n  X,testX,Y,testY = train_test_split(X,Y,test_size=0.7,stratify=Y)\n  #temp1=temp[0:8000]\n  #print(temp1)\n  return X,Y\n  "

In [0]:
#getting the ALgorithms 
from sklearn import linear_model
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

random_f=RandomForestClassifier(n_estimators=100)
logit = linear_model.LogisticRegression()
Lr  = linear_model.LinearRegression()
adaboost = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1), n_estimators=200)
xgb = XGBClassifier()



In [0]:

from sklearn.datasets.samples_generator import make_blobs
from sklearn.metrics import accuracy_score
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from matplotlib import pyplot
from numpy import mean
from numpy import std
from numpy import array
from numpy import argmax
from numpy import tensordot
from numpy.linalg import norm
from scipy.optimize import differential_evolution
from sklearn.model_selection import train_test_split
# fit model on dataset
def fit_model(trainX, trainy):
  trainy_enc = to_categorical(trainy)
  # define model
  model = Sequential()
  model.add(Dense(25, input_dim=2, activation='relu'))
  model.add(Dense(3, activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  # fit model
  model.fit(trainX, trainy_enc, epochs=500, verbose=0)
  return model

# make an ensemble prediction for multi-class classification
def ensemble_predictions(members, weights, testX):
  # make predictions
  yhats = [model.predict_proba(testX) for model in members]
  yhats = array(yhats)
  # weighted sum across ensemble members
  summed = tensordot(yhats, weights, axes=((0),(0)))
  # argmax across classes
  result = argmax(summed, axis=1)
  return result

# # evaluate a specific number of members in an ensemble
def evaluate_ensemble(members, weights, testX, testy):
  # make prediction
  yhat = ensemble_predictions(members, weights, testX)
  # calculate accuracy

  return accuracy_score(testy, yhat)

# normalize a vector to have unit norm
def normalize(weights):
  # calculate l1 vector norm
  result = norm(weights, 1)
  # check for a vector of all zeros
  if result == 0.0:
    return weights
  # return normalized vector (unit norm)
  return weights / result

# loss function for optimization process, designed to be minimized
def loss_function(weights, members, testX, testy):
  # normalize weights
  normalized = normalize(weights)
  # calculate error rate
  return 1.0 - evaluate_ensemble(members, normalized, testX, testy)

'''def count_members(path_Index):
  count=0
  for x in path_Index:
    if x is not -1:
      count=count+1
    else:
      return count
 '''
def memberFun(path_Index,n_members):
  memberList=[]
  for x in path_Index:
    memberList.append(Tree[x]['Algo'])
  
  return memberList
  
def weightCalculation(path_Index):
  # generate 2d classification dataset
  X, y = load_data()#make_blobs(n_samples=1100, centers=3, n_features=2, cluster_std=2, random_state=2)
  # split into train and test

  trainX,testX,trainy,testy = train_test_split(X,y,test_size=0.9,stratify=y)
  print(trainX.shape, testX.shape)
  # fit all models
  n_members =len(path_Index)
  
  members = memberFun(path_Index,n_members)#[fit_model(trainX, trainy) for _ in range(n_members)]
  
  print("*********************")
  print(members)
  print("*********************")
  
  # evaluate each single model on the test set
  for i in members:
    i.fit(trainX,trainy)
  #testy_enc = to_categorical(testy)
  for i in range(n_members):
    test_acc = members[i].score(testX, testy)
    print('Model %d: %.3f' % (i+1, test_acc))
  # evaluate averaging ensemble (equal weights)
  weights = [1.0/n_members for _ in range(n_members)]
  score = evaluate_ensemble(members, weights, testX, testy)
  print('Equal Weights Score: %.3f' % score)
  # define bounds on each weight
  bound_w = [(0.0, 1.0)  for _ in range(n_members)]
  # arguments to the loss function
  search_arg = (members, testX, testy)
  # global optimization of ensemble weights
  print('creating global optimization')
  result = differential_evolution(loss_function, bound_w, search_arg, maxiter=1, tol=1e-7)
  # get the chosen weights
  weights = normalize(result['x'])

  print('Optimized Weights: %s' % weights)
  # evaluate chosen weights
  score = evaluate_ensemble(members, weights, testX, testy)
  print('Optimized Weights Score: %.3f' % score)
  return weights

In [0]:
import pandas as pd
from sklearn.utils import shuffle
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.under_sampling import RandomUnderSampler
def load_data():
  temp = pd.read_csv('/content/bank-additional/bank-additional-full.csv',sep=';')
  df=temp[['euribor3m','cons.price.idx','cons.conf.idx','age','duration','nr.employed','y']]
  #print(df)
  df=shuffle(df)
  Y=df['y']
  Y=Y.eq('yes').mul(1)
  df=df.drop(['y'],axis=1)
  rus = RandomUnderSampler(random_state=42)
  X_res, y_res = rus.fit_resample(df, Y)
  df=pd.DataFrame(X_res,columns=df.columns)
  Y=pd.DataFrame(y_res,columns=['y'])
  
  #df.apply(np.random.shuffle, axis=0)
  #df,Y=shuffle(df,y_res,random_state=42)
  #df=pd.get_dummies(df)
  #print(Y.shape)
  return df,Y
  
  

In [34]:
# Unbalanced data Preprocessing

'''
df['y'].value_counts()
df = df.sample(frac=1,random_state=4)

#undersampling 
yes = df.loc[df['y'] == 'yes']
no = df.loc[df['y'] == 'no'].sample(n=451,random_state=42)
df = pd.concat([yes, no])
df= df.sample(frac=1,random_state=4)
'''

"\ndf['y'].value_counts()\ndf = df.sample(frac=1,random_state=4)\n\n#undersampling \nyes = df.loc[df['y'] == 'yes']\nno = df.loc[df['y'] == 'no'].sample(n=451,random_state=42)\ndf = pd.concat([yes, no])\ndf= df.sample(frac=1,random_state=4)\n"

In [0]:
#X= df.drop(['y'],axis=1)
#X=df
X_,Y=load_data()
dft = pd.concat([X_,Y],axis=1)
dft=shuffle(dft)
Y=dft['y']
X_ = dft.drop(['y'],axis=1)



In [36]:
dft.head()

,euribor3m,cons.price.idx,cons.conf.idx,age,duration,nr.employed,y
3354,4.967,94.465,-41.8,41.0,189.0,5228.1,0
6652,1.244,92.893,-46.2,33.0,2301.0,5099.1,1
4248,1.266,92.893,-46.2,24.0,210.0,5099.1,0
4868,4.968,93.444,-36.1,54.0,429.0,5228.1,1
5684,0.905,94.027,-38.3,64.0,239.0,4991.6,1


In [0]:
#onehot encoding 
dfe=pd.get_dummies(X_)


In [0]:
def multiclass_roc_auc_score(y_test_temp, y_pred_temp, average="micro"):
  #print("inside multiclass_roc_auc_score")
  y_test_temp=list(y_test_temp)
  #print(y_test_temp)
  y_test = label_binarize(y_test_temp,classes=[0,1,2])
  #print("y_test:",y_test)
  y_pred_temp=list(y_pred_temp)
  y_pred = label_binarize(y_pred_temp,classes=[0,1,2])
  #print(y_pred)
  n_class = y_test.shape[1]
  print(n_class)
  avg =0
  for i in range(0, n_class):
    avg= avg+fast_auc(y_test[:,i],y_pred[:,i])
  #print("avg:",avg)
  return avg/n_class

In [0]:

#multiclass_roc_auc_score(VY1, y_pred)

In [0]:
model = [random_f,logit,xgb,adaboost]
name_of_model = ['random_f','logit','xgb','adaboost']

In [0]:
#model = [logit]
#name_of_model = ['logit']

In [42]:
#scaling
'''from sklearn import preprocessing
import copy 
min_max_scaler = preprocessing.MinMaxScaler()
temp1=copy.deepcopy(dfe)

temp1 = pd.DataFrame(min_max_scaler.fit_transform(dfe))

temp1.columns=dfe.columns

dfe=temp1
'''

'from sklearn import preprocessing\nimport copy \nmin_max_scaler = preprocessing.MinMaxScaler()\ntemp1=copy.deepcopy(dfe)\n\ntemp1 = pd.DataFrame(min_max_scaler.fit_transform(dfe))\n\ntemp1.columns=dfe.columns\n\ndfe=temp1\n'

In [0]:
from sklearn.model_selection import train_test_split
XT,VX,YT,VY = train_test_split(dfe,Y,test_size=0.2)

In [0]:
VX1,VX2,VY1,VY2 = train_test_split(VX,VY,test_size=0.5)

In [45]:
VY1.shape

(928,)

In [0]:
def fast_auc(y_true, y_prob):
  try:
      y_true = np.asarray(y_true)
      y_true = y_true[np.argsort(y_prob)]
      #print(y_true)
      nfalse = 0
      auc = 0
      n = len(y_true)
      for i in range(n):
          y_i = y_true[i]
          nfalse += (1 - y_i)
          auc += y_i * nfalse
      auc /= (nfalse * (n - nfalse))
  except:
      print('problem ishre ',nfalse * (n - nfalse))
  return auc

In [47]:
# model X V1 V2 Y
#step 1
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
import numpy as np
import copy
loss_list=[]
path=[]
X=X_.columns
for m in model:
  mod=copy.deepcopy(m)
  #mod=m
  mod.fit(XT,YT)
  Y_pred = mod.predict(VX1)
  #result = cross_val_score(mod, XT, YT, cv=5,scoring='roc_auc')
  try:
    loss=1-fast_auc(VY1,Y_pred)
    #loss = 1-accuracy_score(VY1,Y_pred)
    #loss = roc_auc_score(VY1, y_pred)
    #loss =1-multiclass_roc_auc_score(VY1, Y_pred)
    loss_list.append(loss)
  except:
    print("Something is not right")
print(loss_list)
index =np.argmin(loss_list)
hx = model[index]
loss = loss_list[index]
print(hx," ",loss)



[0.11988381549901239, 0.148962472406181, 0.11671430231207158, 0.11784361566167079]
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)   0.11671430231207158


In [0]:
def min_loss(model,XT,YT,temp_VX):
  loss_list=[]
  for m in model:
    mod=copy.deepcopy(m)
    #mod=m
    mod.fit(XT,YT)
    Y_pred = mod.predict(temp_VX)
    
    try:
      loss_list.append(1-fast_auc(VY1,Y_pred))
      #loss_list.append(1-accuracy_score(VY1,Y_pred))
      #loss_list.append(1-multiclass_roc_auc_score(VY1,Y_pred))
      
    except:
      
      print("Something is not right")
  return np.argmin(loss_list),min(loss_list)
  

In [0]:
def min_loss_cross(model,XT,YT):
  loss_list=[]
  for m in model:
    mod=copy.deepcopy(m)
    result = cross_val_score(mod, XT, YT, cv=5,scoring='roc_auc')

    try:
      loss_list.append(1-result.mean())
      #loss_list.append(1-accuracy_score(VY1,Y_pred))
      
    except:
      print("Something is not right")
  return np.argmin(loss_list),min(loss_list)
  

In [50]:
#########################################################################
#making the tree
#########################################################################
Tree = [-1]*100

Tree[0]={
 'Algo':hx,
 'Split_feature' : -1,
 'left':-1,
 'right':-1,
 'parent':-1,
 'X':X,
 'Y':Y,
 'loss':loss,
 'threshold':-1
}
print(Tree)
#createTree(Tree,0)



[{'Algo': XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1), 'Split_feature': -1, 'left': -1, 'right': -1, 'parent': -1, 'X': Index(['euribor3m', 'cons.price.idx', 'cons.conf.idx', 'age', 'duration',
       'nr.employed'],
      dtype='object'), 'Y': 3354    0
6652    1
4248    0
4868    1
5684    1
2765    0
7789    1
6944    1
2477    0
8575    1
223     0
2652    0
1792    0
6330    1
7676    1
8750    1
2270    0
7056    1
270     0
2250    0
6758    1
1885    0
2455    0
7041    1
5537    1
3187    0
2247    0
4151    0
6497    1
5836    1
       ..
7567    1
1769    0

In [51]:
###################### recursive tree##################
from tqdm import tqdm
# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

def test_split(attribute,threshold,dataset,Y):
  
  temp = pd.concat([dataset,pd.Series(Y)],axis=1)
  left_X,right_X=pd.DataFrame(columns=temp.columns),pd.DataFrame(columns=temp.columns)
  for index,row in temp.iterrows():
    if row[attribute]<threshold:
      left_X.loc[len(left_X)]=row
    else:
      right_X.loc[len(right_X)]=row
  
  return left_X,right_X


import numpy as np
def createTree(Tree,index,features):
  print("creating the node ", index)
  th=[]
  final_loss =1
  final_th=0
  final_Feature='none'
  final_Loss_left=Tree[index]['loss']
  final_Loss_right=Tree[index]['loss']
  final_leftX = Tree[index]['X']
  final_leftY =Tree[index]['Y']
  final_rightX = Tree[index]['X']
  final_rightY =Tree[index]['Y']
  final_left_algo=Tree[index]['Algo']
  final_right_algo=Tree[index]['Algo']
  node = []
  right_temp_X=[]
  left_temp_X=[]
  flagL=1
  flagR=1
  print("Creating the split")
  for i in Tree[index]['X']:
    th=[]
    print("***************",features,"-----------------")
    if i in features:
      continue;
    if len(np.unique(dfe[i]))==1:
      continue
    elif len(np.unique(dfe[i]))==2:
      th=[0.5]
    else:
      #outlier issues here
      temp=[]
      minimum =min(dfe[i])
      maximum = max(dfe[i])
      factor = (maximum-minimum)/10
      #print(minimum,maximum,factor,i)
      #temp = minimum+factor
      #for _ in range(1,8):
       # temp=temp+factor
        #th.append(temp)
      
      #fixing with std and mean
      stdz = dfe[i].std()
      meanz = dfe[i].mean()
      th=np.arange(meanz-stdz/2,meanz+stdz/2,factor)
    print(th)
    for t in th:
      left_X,right_X = test_split(i,t,dfe,Y)    
      left_X=left_X.apply(pd.to_numeric)
      
      right_X=right_X.apply(pd.to_numeric)
      
      left_Y=left_X['y']
      left_X = left_X.iloc[:,:-1]
      right_Y=right_X['y']
      right_X = right_X.iloc[:,:-1]
      
   
      if left_X.shape[0]==0 or right_X.shape[0]==0:
         continue    
 
      if(len(set(left_Y))==1):
         print("Creating the Terminal node with class",left_Y[0])
         Tree[index*2+1]={
          'Algo':'none',
          'Split_feature' : -1,
          'left':-1,
          'right':-1,
          'parent':index,
          'X':left_X,
          'Y' :left_Y[0],
          'loss':0,
          'threshold':t
          }
         flagL=0
      else:
        left_temp_X = left_X.drop([i],axis=1)
        left_temp_VX = VX1.drop([i],axis=1)
        left_algo_index,loss_left = min_loss(model,left_temp_X,left_Y,left_temp_VX)
      if(len(set(right_Y))==1):
        print("Creating the Terminal node with class",right_Y[0])
        Tree[index*2+2]={
          'Algo':'none',
          'Split_feature' : -1,
          'left':-1,
          'right':-1,
          'parent':index,
          'X':right_X,
          'Y' :right_Y[0],
          'loss':0,
          'threshold':t
          }
         
        flagR=0
          
      else:
        right_temp_X = right_X.drop([i],axis=1)
        right_temp_VX = VX1.drop([i],axis=1)
        right_algo_index,loss_right = min_loss(model,right_temp_X,right_Y,right_temp_VX)
      
      
      
      if flagR==1 and flagL==1:
        resultant = min(loss_left,loss_right)
      elif flagR==1 and flagL==0:
        resultant = right_loss
      elif flagR==0 and flagL==1:
        resultant = left_loss
      
      
      print(loss_left,loss_right)
      print(np.unique(left_Y),np.unique(right_Y),t)
        
      print(resultant,i,t)
      if(final_loss>resultant):
        final_th=t
        final_loss = resultant
        final_Feature = i
        final_Loss_left = loss_left        
        final_Loss_right = loss_right
        final_leftX = left_temp_X
        final_leftY =left_Y
        final_rightX = right_temp_X
        final_rightY =right_Y
        final_right_algo = model[right_algo_index]
        final_left_algo = model[left_algo_index]
    print(final_loss," ",final_Feature)
    
    print("------------------------------------particular feature split over ---------------------------------------------------")
  #print(final_leftX,final_rightX)
  #print('(----------------------------------^^^^^^^------------------------)')
  #print(final_Loss_left,final_Loss_right)
  #print(final_th)
  #print(np.unique(final_leftY),np.unique(final_rightY),final_th)
  #print('(----------------------------------^^^^^^^------------------------)')
  print("finish the split")     
  if final_loss<Tree[index]['loss']:
    Tree[index]['Split_feature']=final_Feature
    Tree[index]['threshold']=final_th
    Tree[index]['left']=2*index+1
    Tree[index]['right']=2*index+2
    if flagL == 1:
      #pathL=deepcopy(path)
      #pathL.append(2*index+1)
      Tree[index*2+1]={
          'Algo':final_left_algo,
          'Split_feature' : -1,
          'left':-1,
          'right':-1,
          'parent':index,
          'X':final_leftX,
          'Y' :final_leftY,
          'loss':final_Loss_left,
          'threshold':-1
            }
    if flagR == 1:
      #pathR=deepcopy(path)
      #pathR.append(2*index+2)
      Tree[index*2+2]={
          'Algo':final_right_algo,
          'Split_feature' : -1,
          'left':-1,
          'right':-1,
          'parent':index,
          'X':final_rightX,
          'Y' :final_rightY,
          'loss':final_Loss_right,
          'threshold':-1
            }
    print(final_loss,final_Loss_left,final_Loss_right,final_Feature,Tree[index]['Algo'],final_right_algo,final_left_algo)
    if flagL == 1 and final_Loss_left<Tree[index]['loss']:
      leftf=copy.deepcopy(features)
      leftf.append(final_Feature)
      createTree(Tree,2*index+1,leftf)
    #else:
      #print("$$$$$$$$$$")
      #print(path)
      #print("$$$$$$$$$$")
    if flagR == 1 and final_Loss_right<Tree[index]['loss']:
      rightf=copy.deepcopy(features)
      rightf.append(final_Feature)
      createTree(Tree,2*index+2,rightf)
    #else:
      #print("$$$$$$$$$$")
      #print(path)
      #print("$$$$$$$$$$")
  else:
    #print("$$$$$$$$$$")
    #print(path)
    #print("$$$$$$$$$$")
    if flagL==1:
      Tree[index*2+1]={
          'Algo':'none',
          'Split_feature' : -1,
          'left':-1,
          'right':-1,
          'parent':index,
          'X':-1,
          'Y' :-1,
          'loss':-1,
          'threshold':-1
            }
    if flagR==1:
      Tree[index*2+2]={
          'Algo':'none',
          'Split_feature' : -1,
          'left':-1,
          'right':-1,
          'parent':index,
          'X':-1,
          'Y' :-1,
          'loss':-1,
          'threshold':-1
            }
      
feature=[]
createTree(Tree,0,feature)

creating the node  0
Creating the split
*************** [] -----------------
[2.03752415 2.47862415 2.91972415 3.36082415 3.80192415]
0.2167305681422098 0.2515394446380853
[0. 1.] [0. 1.] 2.0375241541102396
0.2167305681422098 euribor3m 2.0375241541102396
0.19680724991286158 0.2962983617985361
[0. 1.] [0. 1.] 2.4786241541102396
0.19680724991286158 euribor3m 2.4786241541102396
0.1608179388869525 0.2910607644940165
[0. 1.] [0. 1.] 2.9197241541102397
0.1608179388869525 euribor3m 2.9197241541102397
0.22215870802834903 0.27506448239804815
[0. 1.] [0. 1.] 3.3608241541102397
0.22215870802834903 euribor3m 3.3608241541102397
0.22118740560009298 0.2703566864180318
[0. 1.] [0. 1.] 3.8019241541102398
0.22118740560009298 euribor3m 3.8019241541102398
0.1608179388869525   euribor3m
------------------------------------particular feature split over ---------------------------------------------------
*************** [] -----------------
[93.16744585 93.42404585 93.68064585]
0.278215406064831 0.0914093179

In [52]:
Tree


[{'Algo': XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
                colsample_bynode=1, colsample_bytree=1, gamma=0,
                learning_rate=0.1, max_delta_step=0, max_depth=3,
                min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
                nthread=None, objective='binary:logistic', random_state=0,
                reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
                silent=None, subsample=1, verbosity=1),
  'Split_feature': 'age',
  'X': Index(['euribor3m', 'cons.price.idx', 'cons.conf.idx', 'age', 'duration',
         'nr.employed'],
        dtype='object'),
  'Y': 3354    0
  6652    1
  4248    0
  4868    1
  5684    1
  2765    0
  7789    1
  6944    1
  2477    0
  8575    1
  223     0
  2652    0
  1792    0
  6330    1
  7676    1
  8750    1
  2270    0
  7056    1
  270     0
  2250    0
  6758    1
  1885    0
  2455    0
  7041    1
  5537    1
  3187    0
  2247    0
  4151    0
  6497   

In [0]:

import copy
final_path_list=[-1]*20
path=[]
i=0
def createPath(path,index):
  print("inside fun")
  global i
  print(i)
  path_temp=copy.deepcopy(path)
  path_temp.append(index)
  if Tree[index*2+1] is -1 or Tree[index*2+1]['Algo'] is 'none' and Tree[index*2+2] is -1 or Tree[index*2+2]['Algo'] is 'none':
    final_path_list[i]=path_temp 
    i=i+1
    return
    
  if Tree[index*2+1] is not -1 and Tree[index*2+1]['Algo'] is not 'none':
    createPath(path_temp,index*2+1)
  if Tree[index*2+2] is not -1 and Tree[index*2+2]['Algo'] is not 'none':
    createPath(path_temp,index*2+2)


In [54]:
createPath(path,0)

inside fun
0
inside fun
0
inside fun
0
inside fun
1
inside fun
2
inside fun
2
inside fun
3


In [55]:
final_path_list

[[0, 1, 3],
 [0, 1, 4],
 [0, 2, 5],
 [0, 2, 6],
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1]

In [62]:
i=0
weight_list_for_path=[-1]*20
for x in final_path_list:
  if x is not -1:
    weight_list_for_path[i]=weightCalculation(x)
    i=i+1
  else:
    break
print(weight_list_for_path)
  

(928, 6) (8352, 6)
*********************
[XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1), RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=Fal

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


Model 1: 0.867
Model 2: 0.873
Model 3: 0.873
Equal Weights Score: 0.874
creating global optimization
Optimized Weights: [0.16971508 0.36992357 0.46036136]
Optimized Weights Score: 0.874
(928, 6) (8352, 6)
*********************
[XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1), RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
     

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


Model 1: 0.865
Model 2: 0.865
Model 3: 0.865
Equal Weights Score: 0.868
creating global optimization
Optimized Weights: [0.51304961 0.35297746 0.13397294]
Optimized Weights Score: 0.869
(928, 6) (8352, 6)
*********************
[XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1), RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
     

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


Model 1: 0.871
Model 2: 0.869
Model 3: 0.869
Equal Weights Score: 0.873
creating global optimization
Optimized Weights: [0.78000633 0.01845428 0.20153939]
Optimized Weights Score: 0.875
(928, 6) (8352, 6)
*********************
[XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1), RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
     

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


Model 1: 0.875
Model 2: 0.870
Model 3: 0.870
Equal Weights Score: 0.877
creating global optimization
Optimized Weights: [0.52109454 0.3637     0.11520546]
Optimized Weights Score: 0.879
[array([0.16971508, 0.36992357, 0.46036136]), array([0.51304961, 0.35297746, 0.13397294]), array([0.78000633, 0.01845428, 0.20153939]), array([0.52109454, 0.3637    , 0.11520546]), -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]


In [0]:
def getPath(inst):
  instance_path=[]
  instance_path.append(0)
  index=0
  print(inst[Tree[index]['Split_feature']].values[0])
  while True:
    if Tree[index]['Split_feature'] is not -1:
      if inst[Tree[index]['Split_feature']].values[0] <= Tree[index]['threshold'] and Tree[index*2+1]['Algo'] is not 'none':
        instance_path.append(index*2+1)
        index=index*2+1
      else:
        if inst[Tree[index]['Split_feature']].values[0] > Tree[index]['threshold'] and Tree[index*2+2]['Algo'] is not 'none':
          instance_path.append(index*2+2)
          index=index*2+2
        else:
          return instance_path
    else:
      return instance_path
    
  return instance_path

In [0]:
#testing on validation set 2 VX2 VY2
print(Tree[0]['threshold'])
instance_path=[]
i=0
VY_pred=[]
for x in range(0,len(VX2)):
  inst=VX2.iloc[[i]]
  #print(inst)
  instance_path.append(getPath(inst))
  indexOfList=final_path_list.index(instance_path[i])
  wtListForInstance=weight_list_for_path[indexOfList]
  out=[]
  final_out=[]
  for j in range(0,len(instance_path[i])):
    hx=model[model.index(Tree[instance_path[i][j]]['Algo'])]
    w = wtListForInstance[j]
    out.append(w*hx.predict_proba(inst))
  final_out=np.array(out).sum(axis=0)
  VY_pred.append(np.argmax(final_out))
  i=i+1
print(VY_pred)



In [113]:
accuracy_score(VY2,VY_pred)
print(1-fast_auc(VY2,VY_pred))

0.10319042639711862


In [67]:
indexOfList=final_path_list.index(instance_path[i])
wtListForInstance=weight_list_for_path[indexOfList]
print(wtListForInstance)


[0.78000633 0.01845428 0.20153939]


In [0]:
out=[]
for i in range(0,len(instance_path[0])):
  hx=model[model.index(Tree[instance_path[0][i]]['Algo'])]
  w = wtListForInstance[i]
  out.append(w*hx.predict_proba(inst))

In [0]:
final_o=np.array(out).sum(axis=0)

In [87]:
np.argmax(final_o)

1